git clone https://github.com/facebookresearch/fastText.git

cd fastText

make



### make sure you have run ./DATA/preprocess.ipynb first 
### make sure you have cloned and made fasttext
```
mkdir fasttext (already created just cd)
git clone https://github.com/facebookresearch/fastText.git
cd fastTex
make
```
### make sure for first time you run model.preprocess(pre = False ,dim=400, epoch=20 , lr = 0.06 , ws = 10 )

In [4]:
import json
import numpy as np
print(np.__version__)
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import defaultdict
from typing import List,Tuple
import spacy
import os
from gensim.models import KeyedVectors
from scipy.spatial import distance


1.19.2


In [7]:
def softmax(x):
    y = np.exp(x - np.max(x))
    return y / y.sum()

class Fast_text_TF_IDF_IR:
    def __init__(self,t = "lemma" , c_soft = True):
        self.documents = json.load(open("DATA/crawler/data/NLP.json","r"))
        self.t = t
        self.mapping = None
        self.idf = None
        self.train_data_path = None
        if t == "lemma":  
            self.nlp = spacy.load("en_core_web_sm")
            self.tokenizer = lambda s : [token.lemma_ for token in self.nlp(s)]
            self.train_data_path = "./fasttext/fasttext_data.txt"
            self.mapping = json.load(open("DATA/Module_data/abstract_lemma.json","r"))
            self.idf = json.load(open("DATA/Module_data/idf_abstract.json","r"))
        else:
            self.tokenizer = lambda s : [token for token in word_tokenize(s)]
            self.train_data_path = "./fasttext/fasttext_not_lemma_data.txt"
            self.mapping = json.load(open("DATA/Module_data/abstract_not_lemma.json","r"))
            self.idf = json.load(open("DATA/Module_data/idf_abstract_not_lemma.json","r"))
        self.emmbeding = None
        self.mapping = {key : int(self.mapping[key]) for key in self.mapping}
        self.idf = {int(key) : float(self.idf[key]) for key in self.idf}
        self.c_soft = lambda x : x
        if c_soft:
            self.c_soft = lambda x : softmax(x)
        self.doc_emb = {}
        self.dim = 300
    def is_c_(self,w):
        try:
            self.emmbeding[w]
            return True
        except:
            return False

    
    def preprocess(self,pre = False , ws = 5 ,epoch = 20 ,lr = 0.1, dim = 200):
        self.dim = dim
        if not pre:
            os.system("rm ./fasttext/word_embedding.*")
            os.system(f"./fasttext/fastText/fasttext skipgram -dim {dim} -ws {ws} -epoch {epoch} -lr {lr} -input {self.train_data_path} -output ./fasttext/word_embedding")
            os.system("rm ./fasttext/word_embedding.bin")
            self.emmbeding = KeyedVectors.load_word2vec_format("./fasttext/word_embedding.vec")
            for key in self.documents:
                article = self.documents[key]
                abstract = article["abstract"]
                if (type(abstract) == str):
                    try:
                        word = self.tokenizer(abstract)
                        matrix = np.array([self.emmbeding[w] for w in word if self.is_c_(w)]).reshape(-1,self.dim)
                        c = np.array([self.idf.get(self.mapping.get(w,0),0) for w in word if self.is_c_(w)]).reshape(1,-1)
                        c = self.c_soft(c)
                        self.doc_emb[key] = np.matmul(c,matrix).tolist()[0]
                    except:
                        print(key,c)
            open("./fasttext/doc_embedding.json","w").write(json.dumps(self.doc_emb))

        self.emmbeding = KeyedVectors.load_word2vec_format("./fasttext/word_embedding.vec")
        self.doc_emb = json.load(open("./fasttext/doc_embedding.json","r"))
        self.doc_emb = {key : np.array(self.doc_emb[key]).reshape(1,self.dim) for key in self.doc_emb}

    def process_q(self,q : np.array) -> List[Tuple]:
        return sorted([(key,np.abs(distance.cosine(q,self.doc_emb[key]))) for key in self.doc_emb],key = lambda x : x[1])
        

    def query(self, input_string:str , k : int = 10) -> List:
        word = self.tokenizer(input_string.strip().lower())
        matrix = np.array([self.emmbeding[w] for w in word if self.is_c_(w)]).reshape(-1,self.dim)
        c = self.c_soft(np.array([self.idf.get(self.mapping.get(w,0),0) for w in word if self.is_c_(w)]).reshape(1,-1))
        q = np.matmul(c,matrix)[0]
        article_id = self.process_q(q)[:k]
        articles = [self.documents[id[0]] for id in article_id]
        return (article_id,articles)


In [13]:
model = Fast_text_TF_IDF_IR(t = "lemma")
model.preprocess(pre = True ,dim=400, epoch=20 , lr = 0.06 , ws = 10 )

In [35]:
ranking,articles = model.query("performance of NLP models, while alternative approaches for evaluating models either focus on individual tasks or on specific behaviors. Inspired by principles of behavioral testing in software engineering, we introduce CheckList, a task-agnostic methodology for testing NLP models. CheckList includes a matrix of general linguistic capabilities and test types that facilitate comprehensive test ideation, as well as a software tool to generate a large and diverse number of test cases quickly. We illustrate the utility of CheckList with tests for three tasks, identifying critical failures in both commercial and state-of-art models. In a user study, a team responsible for a commercial sentiment analysis model found new and actionable bugs in an extensively tested model. In another user study, NLP practitioners with CheckList created twice as many tests, and found almost three times as many bugs as users without it")
for a,m in zip(ranking,articles):
    print(a)
    print(m["title"])
    print(m["abstract"])

('48709b65320bf979d19c1874aebf6bf8bd85de88', 0.09605803161158266)
Just What do You Think You're Doing, Dave?' A Checklist for Responsible Data Use in NLP
A key part of the NLP ethics movement is responsible use of data, but exactly what that means or how it can be best achieved remain unclear. This position paper discusses the core legal and ethical principles for collection and sharing of textual data, and the tensions between them. We propose a potential checklist for responsible data (re-)use that could both standardise the peer review of conference submissions, as well as enable a more in-depth view of published research across the community. Our proposal aims to contribute to the development of a consistent standard for data (re-)use, embraced across NLP conferences.
('e818c127134e287a4e690d5c1ca0126ff82cfa10', 0.16332067866680544)
Socially Responsible NLP
As language technologies have become increasingly prevalent, there is a growing awareness that decisions we make about our dat